In [ ]:
import numpy as np
import pandas as pd

from bikewheelcalc import BicycleWheel, Hub, Rim
import bikewheellib as bl
from doetools import TensionBucklingExpDOE
# from doetools.wheel_library import create_wheel_from_lm
from shutil import copy2
from imp import find_module

import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
dyn_dir = '../data/doe/tension_buckling_dynamic'

up = 0.001e-3
doe = TensionBucklingExpDOE(out_dir=dyn_dir,
                            opts={'spk_paired': False,
                                  'spk_eltype': 'beam',
                                  'rim_perturb': [0., up, up, up, up],
                                  'sim_time': 1.0,
                                  'sim_temp_factor': 1.5})

R = 0.300
GJ = 25.0
EI1 = 100.
EI2 = GJ / 0.2

wh = BicycleWheel()
wh.hub = Hub(diam1=0.05, width1=0.025, width2=0.025)
wh.rim = Rim(radius=R, area=100e-6,
             I11=GJ / 26.0e9, I22=EI2 / 26.0e9, I33=EI1 / 26.0e9, Iw=0.0 / 69.0e9,
             young_mod=69.0e9, shear_mod=26.0e9)

wh.lace_radial(n_spokes=36, diameter=1.8e-3, young_mod=200e9, offset=0.00)

print bl.calc_buckling_tension(wh)

doe.add_experiment(wh, opts={'jobname': 'Tc_dynamic'})


R = 0.300
GJ = 10.0
EI1 = 100.
EI2 = GJ / 0.1

wh = BicycleWheel()
wh.hub = Hub(diam1=0.05, width1=0.025, width2=0.025)
wh.rim = Rim(radius=R, area=100e-6,
             I11=GJ / 26.0e9, I22=EI2 / 26.0e9, I33=EI1 / 26.0e9, Iw=0.0 / 69.0e9,
             young_mod=69.0e9, shear_mod=26.0e9)

wh.lace_radial(n_spokes=36, diameter=1.8e-3, young_mod=200e9, offset=0.00)

print bl.calc_buckling_tension(wh)

doe.add_experiment(wh, opts={'jobname': 'Tc_dynamic_lo_mu'})



print('\nCreated {0:d} simulations'.format(len(doe.db)))

doe.write_input_files(N_batches=1)
doe.to_csv()

# Copy postprocessing script
copy2(src=find_module('doetools')[1] + '/postproc_tension_exp.py',
      dst=dyn_dir)

In [ ]:
t = pd.read_csv(dyn_dir + '/Tc_dynamic_Pd.csv')
s = pd.read_csv(dyn_dir + '/Tc_dynamic_SE.csv')

t.loc[t['Step'] == 0, 'Strain Energy'] = np.interp(t.loc[t['Step'] == 0, 'Time [s]'],
                                                   s.loc[s['Step'] == 0, 'Time [s]'],
                                                   s.loc[s['Step'] == 0, 'Strain Energy [J]'])

t.loc[t['Step'] == 1, 'Strain Energy'] = np.interp(t.loc[t['Step'] == 1, 'Time [s]'],
                                                   s.loc[s['Step'] == 1, 'Time [s]'],
                                                   s.loc[s['Step'] == 1, 'Strain Energy [J]'])

fig, ax = plt.subplots(ncols=2, figsize=(12, 6))

ax[0].plot(-t['Temperature'], t['Tension [N]'])
ax[1].plot(-t['Temperature'], t['Strain Energy'])